In [11]:
import pandas as pd

In [12]:
PATH = "/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/"

In [13]:
block = pd.read_csv(PATH + "ToDatabase/full_blockgroup.csv")
address_counts = pd.read_csv(PATH + "Processed-Data/queryresults_water/address_count_per_blockgroup_wtr.csv")
address_count_block = pd.read_csv(PATH + "Processed-Data/queryresults_water/address_count_per_blockgroup_housingproject_wtr.csv")
avg_kwh = pd.read_csv(PATH + "Processed-Data/queryresults_water/avggal_blockgroup_wtr.csv")
avg_kwh_block = pd.read_csv(PATH + "Processed-Data/queryresults_water/avg_gal_per_blockgroup_wtr.csv")

In [14]:
df = (block[["total_vacdata", "block_group", "tract"]]).rename(columns = {"tract":"Tract","block_group":"BlockGroup"})

In [15]:
df_full = df.merge(address_counts, on = ["Tract", "BlockGroup"], how = "inner")
df_full = df_full.merge(address_count_block, on = ['Tract', "BlockGroup"], how = "left")
df_full = df_full.merge(avg_kwh, on = ['Tract', "BlockGroup"], how = "left")
df_full = df_full.merge(avg_kwh_block, on = ['Tract', "BlockGroup"], how = "left")

In [16]:
df_full = df_full.fillna(0)

In [17]:
df_full = df_full.rename(columns = {"address_count_x":"utilities_address_count", "address_count_y":"projects_address_count"})

In [18]:
df_full["nonproj_address_count"] = df_full["utilities_address_count"] - df_full["projects_address_count"]

In [19]:
df_full["proj_pct"] = df_full["projects_address_count"] / df_full["utilities_address_count"] * 100

In [20]:
df_full["nonproj_pct"] = 100 - df_full["proj_pct"]

In [21]:
df_full = df_full.drop("address_count", axis = 1)

In [22]:
df_full = df_full.rename(columns = {"mean_gal":"project_mean_gal", "avg_gal":"nonproj_mean_gal"}).drop("numrecords", axis = 1)

In [23]:
df_full = df_full.rename(columns = {"total_vacdata":"total_properties"})

In [24]:
cols = list(df_full)
cols.insert(0, cols.pop(cols.index('Tract')))
cols.insert(1, cols.pop(cols.index("BlockGroup")))
cols.insert(5, cols.pop(cols.index("nonproj_address_count")))
cols.insert(6, cols.pop(cols.index("proj_pct")))
cols.insert(7, cols.pop(cols.index("nonproj_pct")))
df_full = df_full.loc[:, cols]

In [25]:
sum_props = df_full["total_properties"].sum()
sum_utilities = df_full["utilities_address_count"].sum()
sum_projects = df_full["projects_address_count"].sum()
sum_nonproj = df_full["nonproj_address_count"].sum()

In [26]:
df_full = df_full.append({"total_properties" : sum_props, "utilities_address_count" : sum_utilities,
                         "projects_address_count" : sum_projects, "nonproj_address_count" : sum_nonproj}, ignore_index =  True)

In [28]:
df_full.tail()

,Tract,BlockGroup,total_properties,utilities_address_count,projects_address_count,nonproj_address_count,proj_pct,nonproj_pct,project_mean_gal,nonproj_mean_gal
65,11400.0,1.0,427.0,323.0,10.0,313.0,3.095975,96.904025,4.283166,5.760381
66,11400.0,2.0,181.0,117.0,2.0,115.0,1.709402,98.290598,7.802589,6.713609
67,11600.0,1.0,649.0,205.0,0.0,205.0,0.000000,100.000000,0.000000,6.719882
68,20300.0,2.0,616.0,1.0,0.0,1.0,0.000000,100.000000,0.000000,32.867470
69,NaN,NaN,41182.0,25945.0,482.0,25463.0,NaN,NaN,NaN,NaN


In [29]:
df_full.loc[69, "Tract"] = "Totals"
df_full.loc[69, "proj_pct"] = sum_projects/sum_utilities*100
df_full.loc[69, "nonproj_pct"] = sum_nonproj/sum_utilities*100
df_full.loc[69, "project_mean_gal"] = 11.85

In [30]:
df_full

,Tract,BlockGroup,total_properties,utilities_address_count,projects_address_count,nonproj_address_count,proj_pct,nonproj_pct,project_mean_gal,nonproj_mean_gal
0,100,1.0,433.0,250.0,10.0,240.0,4.000000,96.000000,6.212804,5.953043
1,100,2.0,175.0,118.0,1.0,117.0,0.847458,99.152542,4.168675,5.977296
2,100,3.0,397.0,345.0,7.0,338.0,2.028986,97.971014,4.514132,6.156197
3,100,4.0,938.0,302.0,6.0,296.0,1.986755,98.013245,4.872950,9.742876
4,100,5.0,493.0,330.0,6.0,324.0,1.818182,98.181818,7.241379,6.232909
5,200,1.0,352.0,286.0,12.0,274.0,4.195804,95.804196,4.572704,5.890435
6,200,2.0,170.0,174.0,6.0,168.0,3.448276,96.551724,4.407917,6.017323
7,200,3.0,560.0,255.0,26.0,229.0,10.196078,89.803922,5.725868,5.771035
8,200,4.0,244.0,174.0,8.0,166.0,4.597701,95.402299,3.752983,7.755489
9,400,1.0,738.0,470.0,2.0,468.0,0.425532,99.574468,25.920375,6.044737


In [32]:
df_full.to_csv(PATH + 'Tables/water.csv', index=False)